<!-- サンチャゴの大暴落で勝つから引用 -->
<!-- 移動平均線からの乖離が大きい銘柄に対してエントリー -->

In [23]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("../")
from common.evaluate.evaluate import evaluation_index
import codecs
from joblib import Parallel, delayed
import pandas_datareader.data as web
import tqdm
import matplotlib.pyplot as plt


In [24]:
candle_data_path = "../data/candle_data/TOPIX"
os.makedirs(candle_data_path, exist_ok=True)

In [25]:

# files = os.listdir('../data/raw_data/candle_data/TOPIX/')
# for file in files:
#     data = pd.read_csv('../data/raw_data/candle_data/TOPIX/'+file,index_col=0)
#     # print(data.iloc[0])
#     print(file)
#     data = data.rename(columns={'Open': 'open', 'High': 'high','Low': 'low','Close': 'cl','Adj Close':'close','Volume': 'volume'})

#     data['price'] = data['close'].shift(-1) 
#     for sma in [5,25,45,75]:
#         data[f"sma{sma}"] = data['close'].rolling(sma).mean()
#         data[f'sma_close_ratio_{sma}'] = (data['close'] / data[f"sma{sma}"] - 1) * 100
#     data.to_csv(candle_data_path+'/'+file)
#     # display(data.iloc[0])


In [26]:
code_name = 'トヨタ自'
df = pd.read_csv(f'../data/candle_data/TOPIX/{code_name}.csv',index_col = 0)
# df = df['2020-01-04':]
display(df)

,high,low,open,cl,volume,close,price,sma5,sma_close_ratio_5,sma25,sma_close_ratio_25,sma45,sma_close_ratio_45,sma75,sma_close_ratio_75
Date,,,,,,,,,,,,,,,
2019-01-04,1272.000000,1232.199951,1236.000000,1269.199951,50059500.0,904.492859,932.998657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-07,1324.400024,1295.599976,1299.000000,1309.199951,38386000.0,932.998657,944.401123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08,1337.400024,1316.800049,1322.000000,1325.199951,39896500.0,944.401123,951.670166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-09,1343.000000,1325.800049,1342.400024,1335.400024,27472000.0,951.670166,955.518494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-10,1342.800049,1325.400024,1326.000000,1340.800049,31702000.0,955.518494,972.194397,937.81626,1.887602,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-10,2204.000000,2155.000000,2190.000000,2178.500000,28054600.0,2178.500000,2082.000000,2232.60000,-2.423184,2185.64,-0.326678,2115.255556,2.989920,2162.600000,0.735226
2022-05-11,2210.000000,2050.500000,2205.500000,2082.000000,64821200.0,2082.000000,2050.000000,2201.90000,-5.445297,2180.02,-4.496289,2116.222222,-1.617137,2158.053333,-3.524164
2022-05-12,2114.500000,2011.500000,2050.000000,2050.000000,44104700.0,2050.000000,2055.000000,2166.30000,-5.368601,2173.82,-5.695964,2116.288889,-3.132318,2153.073333,-4.787265


In [27]:
def func(file,date,path,sma):
    try:
        data = pd.read_csv(path+'/'+file,index_col=0)
        # display(data)

        sma_close_ratio = data.loc[date,f'sma_close_ratio_{sma}']
        price = data.loc[date,'price']
        code = file
        # print(file)
        
    except:
        # print(file+':error')
        sma_close_ratio = 0
        price = 0
        code = file

    return sma_close_ratio, price, code



In [28]:
def calcurate_prediction(date, sma):   
    files = os.listdir(candle_data_path)
    # res = Parallel(n_jobs=6)(delayed(func)(file, date,candle_data_path,sma) for file in files[:5])
    # result = pd.DataFrame(res,
    #               columns=['sma_close_ratio', 'price', 'code'])

    sma_close_ratio_list = []
    price_list = []
    code_list = []
    for file in files:
        sma_close_ratio, price, code = func(file,date,candle_data_path,sma)
        sma_close_ratio_list.append(sma_close_ratio)
        price_list.append(price)
        code_list.append(code)
    result = pd.DataFrame(
        {
            "sma_close_ratio": sma_close_ratio_list,
            "price": price_list,
            "code": code_list,
        }
    )

    result = result.sort_values('sma_close_ratio',ascending=True)
    # display(result)

    return result.iloc[0]


In [29]:
def backtest(df,exit_range,sma,entry_threshold):
    code = ''
    df = df.rename(columns={'Open': 'open', 'High': 'high','Low': 'low','Close': 'close','Volume': 'volume'})
    n = df['close'].values.size
    y = df['close'].values.copy() * 0.0
    poss = df['close'].values.copy() * 0.0
    ret = 0.0
    pos = 0.0
    buy_entry_price = 1.0
    sell_entry_price = 1.0
    buy_entry_prices = np.full((n), np.nan)
    sell_entry_prices = np.full((n), np.nan)
    buy_exit_prices = np.full((n), np.nan)
    sell_exit_prices = np.full((n), np.nan)
    for i in tqdm.tqdm(range(len(df))):
        prev_pos = pos
        date = df.index[i]

        # Exit of long
        if prev_pos == 1 and (i - entry_date) >= exit_range:
            path = "../data/candle_data/TOPIX"
            data = pd.read_csv(path+'/'+code,index_col=0)
            data = data.rename(columns={'Open': 'open', 'High': 'high','Low': 'low','Close': 'close','Volume': 'volume'})

            data['price'] = data['close'].shift(-1) 
            sell_exit_price = data.loc[date,'price']
            sell_exit_prices[i] = data.loc[date,'price']
            ret += (sell_exit_price / buy_entry_price - 1)
            pos -= 1

        # entry
        if prev_pos == 0:
            result = calcurate_prediction(date, sma)
            if result[f'sma_close_ratio'] <= entry_threshold:
                buy_entry_price = result['price']
                buy_entry_prices[i] = result['price']
                code = result['code']
                entry_date = i
                pos += 1

        y[i] = ret
        poss[i] = pos

    return y, poss, buy_entry_prices, sell_entry_prices, buy_exit_prices, sell_exit_prices


exit_range = 5
sma = 25
entry_threshold = -20

for exit_range in [1,3,5]:
    for sma in [5,10,25,45,75]:
        for entry_threshold in [-5,-10,-15,-20,-30]:
            try:
                print('==================')
                print(exit_range, sma, entry_threshold)

                cumulative_return, possition, buy_entry_prices, sell_entry_prices, buy_exit_prices, sell_exit_prices = backtest(df,exit_range,sma,entry_threshold)
                df_result =  pd.DataFrame(
                    {
                        "cumulative_return": cumulative_return,
                        "position": possition,
                        "buy_entry_price": buy_entry_prices,
                        "sell_entry_price": sell_entry_prices,
                        "buy_exit_price": buy_exit_prices,
                        "sell_exit_price": sell_exit_prices,
                    },
                    index=df.index
                )
                result_dict = evaluation_index(df_result)
            except:
                print('error')
                continue


df_result['cumulative_return'].plot()
plt.title('cumulative_return')
plt.show()

print('ポジション推移です。変動が細かすぎて青色一色になっていると思います。')
print('ちゃんと全ての期間でトレードが発生しているので、正常です。')
df_result['position'].plot()
plt.title('position')
plt.show()

print('ポジションの平均の推移です。どちらかに偏りすぎていないかなどを確認できます。')
df_result['position'].rolling(1000).mean().plot()
plt.title('position_average')
plt.show()

print('取引量(ポジション差分の絶対値)の累積です。')
print('期間によらず傾きがだいたい同じなので、全ての期間でちゃんとトレードが行われていることがわかります。')
df_result['position'].diff(1).abs().dropna().cumsum().plot()
plt.title('cumulative_position')

1 5 -5


  5%|▍         | 40/815 [00:28<08:34,  1.51it/s]